## Salary Prediction

### Table of Contents

1. [Problem Statement](#Problem-Statement)
2. [Libraries](#Libraries)
3. [Data Loading](#Data-Loading)
4. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
   - [Dataset Summary](#Dataset-Summary)
   - [Dataset Distribution](#Dataset-Distribution)
   - [Data Quality Assessment](#Data-Quality-Assessment)
     - [Missing Values](#Missing-Values)
     - [Duplicates](#Duplicates)
     - [Outliers](#Outliers)
5. [Data Preprocessing](#Data-Preprocessing)
6. [Model Building, Training and Hyperparameter Tuning](#Model-Building-and-Training)
     1. [Model 1: Random Forest](#Model-1:-Random-Forest)
     2. [Model 2: Logistic Regression](#Model-2:-Logistic-Regression)
7. [Model Evaluation](#Model-Evaluation)
8. [Results and Insights](#Results-and-insights)
9.  [Future Work](#Future-Work)
10. [Conclusion](#Conclusion)


#### Problem Statement

#### Libraries

In [1]:
import pandas as pd
import plotly.express as px

#### Data Loading

In [ ]:
salary_df = pd.read_csv("data/salary_data.csv")